# Homework 5

## FINM 35700 - Spring 2023

### UChicago Financial Mathematics

* Alex Popovici
* alex.popovici@uchicago.edu

This homework relies on following symbology & data files, as of 2023-05-12.

US government bonds:
- the US government bonds symbology file `govt_symbology`, 
- the "on-the-run" treasuries data file `govt_on_the_run`,
- the US Government bonds market data file `govt_market_prices_eod`,

SOFR OIS swaps:
- the SOFR OIS swaps symbology file (for hyg basket) `sofr_swaps_symbology`,
- the SOFR OIS swaps market data file `sofr_swaps_market_data_eod`,

CDX Index:
- the CDX Index symbology file (for hyg basket) `cdx_symbology`,
- the CDX Index basket composition file `cdx_basket_composition`,

HYG ETF:
- the corporate  bonds symbology file (for HYG basket) `hyg_corp_symbology`,
- the HYG ETF basket composition file  `hyg_basket_composition`,


## Scoring: Total of 100 points

| Problem | Points |
|---------|--------|
| 1       | 10     |
| 2       | 10     |
| 3       | 15     |
| 4       | 30     |
| 5       | 35     |

# Problem 1: Fixed rate Bond prices and sensitivities (bond yield model)

## a. True or False (fixed rate bond prices)

1. Fixed rate bond price is increasing in yield
2. Fixed rate bond price is increasing in coupon
3. Fixed rate bond price is increasing in bond maturity
4. Fixed rate callable bond prices are higher or equal to their "bullet" (non-callable) version.

## b. True or False (fixed rate bond yields)

1. Fixed rate bond yield is increasing in interest rate
1. Fixed rate bond yield is increasing in credit spread
2. Fixed rate bond yield is increasing in coupon
3. Fixed rate bond yield is increasing in bond maturity
4. Fixed rate callable bond yields are lower or equal to their "bullet" (non-callable) version.

## c. True or False (fixed rate bond durations)

1. Fixed rate bond duration is increasing with yield
2. Fixed rate bond duration is increasing in coupon
3. Fixed rate bond duration is increasing with bond maturity
4. Fixed rate callable bond durations are higher or equal to their "bullet" (non-callable) version .


## d. True or False (fixed rate bond convexities)

1. Fixed rate bond convexity is increasing with yield
2. Fixed rate bond convexity is increasing in coupon
3. Fixed rate bond convexity is increasing with bond maturity
4. Fixed rate callable bond convexities are higher or equal to their "bullet" (non-callable) version .


# Problem 2: Credit Default Swaps (hazard rate model)
## a. True or False (CDS Premium Leg PV)

1. CDS premium leg PV is increasing in CDS Par Spread
2. CDS premium leg PV is increasing in interest rate
2. CDS premium leg PV is increasing in hazard rate
3. CDS premium leg PV is increasing in recovery rate
4. CDS premium leg PV is increasing in coupon
5. CDS premium leg PV is increasing in CDS maturity


## b. True or False (CDS Default Leg PV)

1. CDS default leg PV is increasing in CDS Par Spread
2. CDS default leg PV is increasing in interest rate
3. CDS default leg PV is increasing in hazard rate
4. CDS default leg PV is increasing in recovery rate
5. CDS default leg PV is increasing in coupon
6. CDS default leg PV is increasing in CDS maturity

## c. True or False (CDS PV)


1. CDS PV is increasing in CDS Par Spread
2. CDS PV is increasing in interest rate
3. CDS PV is increasing in hazard rate
4. CDS PV is increasing in recovery rate
5. CDS PV is increasing in coupon
6. CDS PV is increasing in CDS maturity

## d. True or False (CDS Par Spread)


1. CDS Par Spread is increasing in interest rates
2. CDS Par Spread is increasing in hazard rate
3. CDS Par Spread is increasing in recovery rate
4. CDS Par Spread is increasing in coupon
5. CDS Par Spread is increasing in CDS maturity

# Problem 3: Perpetual CDS
We are interested in a perpetual CDS contract (infinite maturity) on a face notional of $100, flat interest rate of 4% and coupon of 5% (quarterly payments).

For simplicity, we assuming a flat hazard rate of 1% per annum, a rcovery of 40%, T+0 settlement and zero accrued.

## a. What is the fair value of the CDS premium leg?


## b. Compute the fair value of the CDS default leg and the CDS Par Spread.

## c. Compute the following CDS risk sensitivities:
- IR01
- HR01
- CS01 and
- REC01

using the scenario method.


## d. At what time to maturity T does the (implied) default probability over next 5 years drop to 1%?

\begin{align}
\mathbb{P} \left(\tau \in [T, T+5] \right) = 1/100
\end{align}


# Problem 4: CDS Intrinisc Index analysis on CDX IG 5Y


In [2]:
import QuantLib as ql
import numpy as np
import pandas as pd
import datetime as dt

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Use static calculation/valuation date of 2023-05-12, matching data available in the market prices EOD file
calc_date = ql.Date(12, 5, 2023)
ql.Settings.instance().evaluationDate = calc_date

## a. Load and explore the CDX IG 5Y symbology, basket composition and market data

Load the `cdx_symbology` Excel file into a dataframe. Identify the CDX IG 5Y Series 40 index.

Load the `cdx_basket_composition` Excel file into a dataframe. It contains the CDX IG 5Y Series 40 basket composition and CDS Par Spread quotes as of 2023-05-12.

How many CDS contracts are in the CDX index basket? What are the contractual specs and weights of each CDS? Remember that each CDS is identical to the CDX index and the basket has uniform weights.

Compute mean, median and standard deviation of CDS Par Spreads in the basket.

Sort the basket dataframe by CDS Par Spreads and plot the chart of CDS Par Spreads.

## b. Calibrate the US SOFR yield curve (via bootstrapping)
Load the `sofr_swap_symbology` Excel file into a dataframe.

Load the `sofr_swaps_market_data_eod` Excel file into a dataframe. Print all dates available.

Follow Homework 3, Problem 3 and calibrate the SOFR discount curve as of 2023-05-12. You can use the SOFR calibration function below.

In [5]:
def calibrate_sofr_curve_from_frame(
        calc_date: ql.Date,
        sofr_details: pd.DataFrame,
        rate_quote_column: str):
    '''Create a calibrated yield curve from a SOFR details dataframe which includes rate quotes.
    '''
    ql.Settings.instance().evaluationDate = calc_date

    # Sort dataframe by maturity
    sorted_details_frame = sofr_details.sort_values(by='term')    
    
    # settle_days
    settle_days = 2
    
    # For US SOFR OIS Swaps 
    day_count = ql.Actual360()

    # For US SOFR Swaps     
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
    
    sofr_helpers = []
    
    for index, row in sorted_details_frame.iterrows():
        sofr_quote = row[rate_quote_column]
        tenor_in_years = row['term']
        sofr_tenor = ql.Period(tenor_in_years, ql.Years)
        
        # create sofr_rate_helper
        sofr_helper = ql.OISRateHelper(settle_days, sofr_tenor, ql.QuoteHandle(ql.SimpleQuote(sofr_quote/100)), ql.Sofr())
                        
        sofr_helpers.append(sofr_helper)
        
    sofr_yield_curve = ql.PiecewiseLinearZero(settle_days, calendar, sofr_helpers, day_count)
    sofr_yield_curve.enableExtrapolation()
    
    return sofr_yield_curve

## c. Compute the intrinsic Premium Leg PV of the CDX index basket

For each instrument in the basket, create the CDS object and calibrate the corresponding hazard rate / survival probability curve.

Compute the Premium Leg PV for each CDS and determine the intrinsic Premium Leg PV of the basket (using basket weights).

## d. Compute the intrinsic Default Leg PV of the CDX index basket

Compute the Default Leg PV for each CDS and determine the intrinsic Default Leg PV of the basket (using basket weights).

## e. Compute the intrinsic PV and Par Spread of the CDX index basket

Compute the intrinsic index PV. 

Compute the CDX instrinsic index Par Spread (using ratio of CDX default vs premium legs).

# Problem 5: Credit ETF analysis on HYG

## a. Load and explore the HYG basket composition and market data

Load the `hyg_basket_composition` Excel file into a dataframe. It contains the HYG basket constituent face notionals, weights and yields-to-maturities as of 2023-05-12.

Load the `hyg_corp_symbology` Excel file into a dataframe. It contains the corporate bond details for HYG constituents.

How many corporate bonds are in the HYG basket?  What are the average and median face notionals for a bond?

How many unique tickers are in the HYG basket? What are the average and median face notionals for a ticker?

Compute mean, median and standard deviation of yields-to-maturity of bonds in the basket.

## b. Compute the NAV of the HYG basket and the intrinsic price of one ETF share.

Create the bond objects for all constituents of HYG. Compute the dirty price for each bond (from yield-to-maturity).

Aggregate the ETF NAV value (intrisic value of bond basket) as the weighted sum of dirty prices times basket weights. Keep in mind that the resulting ETF NAV will be on a face of $100, since the basket face notional weights add up to 100 percent.

Compute the intrinisc market capitalization of the HYG ETF by scaling the ETF NAV price to the ETF total face notional. 

Divide by 195,800,000 (the number of ETF shared outstanding as of 2023-05-12) to obtain the intrinsic price of one HYG ETF share.

## c. Compute the ETF yield using the ACF (Aggregated Cash-Flows) method

Create the bond objects for all constituents of HYG. 

Write a function that computes the ETF NAV for a given flat yield y.

Solve for 

ETF_NAV(yield) = ETF_NAV_Price 

to obtain the ETF yield.





## d. Compute the ETF DV01, Duration and Convexity

Treat the ETF basket as a synthetic bond.

Use +/- 1 bp scenarios in ETF yield space to compute the ETF DV01, Duration and Convexity.


## e. Compute the ETF z-spread

Load and calibrate the US Treasury discount curve as of 2023-05-12. Follow Homework 2, Problem 2.

Write a function that computes the ETF NAV for a given flat z-spread (parallel shift on the top of the US Treasury yield curve)

Solve for 

ETF_NAV(z-spread) = ETF_NAV_Price 

to obtain the ETF z-spread.

Use Section 9 "Analytical Duration, Convexity and Z-Spread" of the QuantLib as a template for applying the flat z-spread to the US Treasury yield curve.